In [94]:
import gym
import torch
import torch.nn as nn
import numpy as np
import random
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import copy

In [19]:
env = gym.make('FrozenLake-v0')
env = env.unwrapped

In [38]:
a = set()
a.add((2,3))
a.add((4,5))
a.add((4,6))
for c, d in a:
    print (c + d)

9
5
10


In [89]:
Q = nn.Sequential(
            nn.Linear(10, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
            )

v = torch.zeros(10, dtype=torch.float)
v[1] = 2
Q([v,v])

AttributeError: 'list' object has no attribute 'dim'

In [104]:
Q = nn.Sequential(
            nn.Linear(2, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
            )
QHat = copy.deepcopy(Q)
v = torch.zeros(2, dtype=torch.float)
print(Q(v))
Q(v).max()

tensor([-0.1345,  0.4005,  0.2612, -0.2137,  0.1535, -0.0513,  0.1094,  0.1190,
         0.3438, -0.0515], grad_fn=<ThAddBackward>)


tensor(0.4005, grad_fn=<MaxBackward1>)

In [83]:
miniBatchSize = 1000
N = 1000 

In [87]:
Y = torch.zeros(10, dtype=torch.float)
Y

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [142]:
def main(nEpisode=100, gamma = 0.99, epsilon0 = 0.9, maxIter = 200):
    epsilon = epsilon0
    buffer = set()
    loss_fn = torch.nn.MSELoss()
    Q = nn.Sequential(
            nn.Linear(2, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.ReLU()
            )
    QHat = copy.deepcopy(Q)
    for step in range(nEpisode):
        print(step)
        obs = env.reset()
        for _ in range(maxIter):
            epsilon = 0.99*epsilon
            if np.random.random() < epsilon:
                action = env.action_space.sample()
            else:
                action = 0
                M = Q(torch.FloatTensor([obs, action]))
                for i in range(env.action_space.n):
                    if Q(torch.FloatTensor([obs, i])) > M:
                        action = i
                        M = Q(torch.FloatTensor([obs, i]))
            obsNext, reward, done , _ = env.step(action)
            buffer.add((obs, action, reward, done, obsNext))
            obs = obsNext
            minibatch = random.sample(buffer, min(len(buffer), miniBatchSize))
            optimizer = torch.optim.SGD(Q.parameters(), lr = 0.01, momentum = 0.9)
            for o, a, r, d, oN in minibatch:
                M = QHat(torch.FloatTensor([oN, action]))
                for i in range(env.action_space.n):
                    if QHat(torch.FloatTensor([oN, i])) > M:
                        M = QHat(torch.FloatTensor([oN, i]))
                output = Q(torch.FloatTensor([o, a]))
                target = torch.FloatTensor([r if d else r + gamma * M])
                optimizer.zero_grad()
                loss = loss_fn(output, target)
                loss.backward()
                optimizer.step()   
            if step % N == 0:
                QHat = copy.deepcopy(Q)        
            if done:
                break
    return Q

In [143]:
Q = main(20)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [144]:
def transform(Q, s, a):
    return float(Q(torch.FloatTensor([s, a])))

In [145]:
q_table = np.ones((env.observation_space.n, env.action_space.n))

for i in range(env.observation_space.n):
    for j in range(env.action_space.n):
        q_table[i, j] = transform(Q, i, j)

In [146]:
def testPolicy (q_table, nEpisode = 2000):
    success = 0
    for _ in range(nEpisode):
        t = 0
        observation = env.reset()
        done  = False
        actionTable = np.argmax(q_table, axis = 1)
        while not done and t < 200:
            action = actionTable[observation]
            observation, reward, done, info = env.step(action)
            t += 1

        if reward == 1:
            success += 1
    return success / nEpisode

In [147]:
testPolicy(q_table)

0.0

In [148]:
q_table

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])